In [1]:
# %% ========================================
# 0. ライブラリ & データ読み込み
# ========================================
import pandas as pd
import numpy as np
import os

# 00 の出力（クリーンデータ）
CLEAN_PATH = "/workspace/data/00/StatsAllGroup_cleaned.parquet"

# 01_exploration の出力（選手プロファイル）
PLAYER_PROFILE_PATH = "/workspace/data/01/player_final_categories.parquet"

df_clean = pd.read_parquet(CLEAN_PATH)
profiles = pd.read_parquet(PLAYER_PROFILE_PATH)

df_clean["date_"] = pd.to_datetime(df_clean["date_"])


print(f"✅ Loaded cleaned data: {df_clean.shape[0]} rows, {df_clean['athlete_name'].nunique()} athletes")
print(f"✅ Loaded player profiles: {profiles.shape[0]} rows")

# ---- 01 で full 判定になった選手だけを対象にする ----
full_players = profiles[profiles["final_category"] == "full"]["athlete_name"].unique()
df_valid = df_clean[df_clean["athlete_name"].isin(full_players)].copy()

print(f"🎯 Using FULL players only: {len(full_players)} athletes")
print(f"   -> Filtered data: {df_valid.shape[0]} rows, {df_valid['athlete_name'].nunique()} athletes")



✅ Loaded cleaned data: 67994 rows, 69 athletes
✅ Loaded player profiles: 57 rows
🎯 Using FULL players only: 34 athletes
   -> Filtered data: 46432 rows, 34 athletes


In [2]:
print("\n🔍 データ型ごとの列名一覧:")

dtype_map = {}
for col, dtype in df_valid.dtypes.items():
    dtype_map.setdefault(str(dtype), []).append(col)

for dtype, cols in dtype_map.items():
    print(f"\n--- {dtype} ({len(cols)} columns) ---")
    for c in cols:
        print(" -", c)


🔍 データ型ごとの列名一覧:

--- datetime64[ns] (3 columns) ---
 - date_
 - start_dt
 - end_dt

--- float64 (50 columns) ---
 - session_duration_sec
 - start_time
 - end_time
 - total_duration
 - total_distance
 - max_heart_rate
 - mean_heart_rate
 - min_heart_rate
 - max_vel
 - velocity_band1_total_distance
 - velocity_band2_total_distance
 - velocity_band3_total_distance
 - velocity_band4_total_distance
 - velocity_band5_total_distance
 - velocity_band6_total_distance
 - gen2_velocity_band1_total_effort_count
 - gen2_velocity_band2_total_effort_count
 - velocity2_band6_total_distance
 - velocity2_band7_total_distance
 - velocity2_band8_total_distance
 - heart_rate_band2_total_duration
 - heart_rate_band3_total_duration
 - total_player_load
 - total_jumps
 - total_dive_load_centre
 - total_dive_load_left
 - total_dive_load_right
 - dive_right_count
 - dive_left_count
 - dive_centre_count
 - total_dives_centre
 - total_dives_left
 - total_dives_right
 - total_time_to_feet_centre
 - total_time_to_f

In [3]:
# ============================================================
# 日次集計：df_valid（セッション単位） → daily_raw（1日1行）
# ============================================================

df = df_valid.copy()

# --------------------------------------------
# (既存コード) Match Day 情報の付与
# --------------------------------------------
df["is_match_day"] = df["activity_name"].str.contains(
    r"vs|VS|tm|TM|match|試合", case=False, na=False
).astype(int)

# ========== 追加推奨コード ここから ==========
# 安全策：is_match_day が数値型でない場合、強制的に int に変換する
if not pd.api.types.is_numeric_dtype(df["is_match_day"]):
    print("⚠️ is_match_day列が数値型ではありませんでした。int型に変換します。")
    # エラーハンドリング：変換できない値（文字列など）がある場合は0にする等の処理
    df["is_match_day"] = pd.to_numeric(df["is_match_day"], errors='coerce').fillna(0).astype(int)
else:
    print("✅ is_match_day列は正常な数値型です。")
# ========== 追加推奨コード ここまで ==========


# ------------- 1. 集計対象の自動判別 ------------- #

numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns.tolist()

# 集計に使わない列を除外
exclude_cols = ["start_time", "end_time"]  # total_duration は残す
numeric_cols = [c for c in numeric_cols if c not in exclude_cols]

agg_dict = {}
for col in numeric_cols:
    if col == "mean_heart_rate":
        agg_dict[col] = "mean"
    elif col in ["max_heart_rate", "max_vel"]:
        agg_dict[col] = "max"
    elif col == "min_heart_rate":
        agg_dict[col] = "min"
    else:
        agg_dict[col] = "sum"

# ← これを追加！
agg_dict["is_match_day"] = "max"

# --------------------------------------------
# Match Day の日付一覧（各選手ではなくチーム全体）
# --------------------------------------------
match_days = (
    df[df["is_match_day"] == 1]["date_"]
      .drop_duplicates()
      .sort_values()
      .tolist()
)


daily_raw = (
    df.groupby(["athlete_name", "date_"])
      .agg(agg_dict)
      .reset_index()
      .sort_values(["athlete_name", "date_"])
)

# --------------------------------------------
# Match Day からの相対日数 md_offset を付与（完全に安全な版）
# --------------------------------------------
daily_raw = daily_raw.sort_values(["athlete_name", "date_"])

if len(match_days) == 0:
    daily_raw["md_offset"] = np.nan
else:
    # すべて numpy datetime64[D] に変換
    md_array = pd.to_datetime(match_days).to_numpy().astype("datetime64[D]")
    date_array = daily_raw["date_"].dt.to_pydatetime()   # Python datetime に変換 → numpy

    # numpy datetime64[D] にキャスト
    date_array = pd.to_datetime(date_array).to_numpy().astype("datetime64[D]")

    # 差分行列：(n_rows, n_match_days)
    diff_matrix = (date_array[:, None] - md_array[None, :]).astype("timedelta64[D]").astype(int)

    # 絶対値が最小の（最も近い試合）を選択
    idx_min = np.abs(diff_matrix).argmin(axis=1)
    md_offset = diff_matrix[np.arange(len(diff_matrix)), idx_min]

    daily_raw["md_offset"] = md_offset




# --------------------------------------------
# md_phase のカテゴリ化（MD, MD+1, MD+2, …）
# --------------------------------------------
def to_md_phase(x):
    if pd.isna(x): return "NONE"
    if x == 0:  return "MD"
    if x == 1:  return "MD+1"
    if x == 2:  return "MD+2"
    if x == -1: return "MD-1"
    if x == -2: return "MD-2"
    return "OTHER"

daily_raw["md_phase"] = daily_raw["md_offset"].apply(to_md_phase)


print("\n✅ 日次集計完了！")
print("shape:", daily_raw.shape)
print(daily_raw.head())

# ざっくり一人分だけ確認
print("\n🔍 例として1人目の選手の最初の5行を確認：")
example_player = daily_raw["athlete_name"].iloc[0]
print("TARGET_PLAYER:", example_player)
print(daily_raw[daily_raw["athlete_name"] == example_player].head())


✅ is_match_day列は正常な数値型です。

✅ 日次集計完了！
shape: (12083, 63)
  athlete_name      date_  session_duration_sec  total_duration  \
0    Bunta INO 2023-01-12               8089.00         8089.00   
1    Bunta INO 2023-01-13               4952.00         4952.00   
2    Bunta INO 2023-01-14               6672.00         6672.00   
3    Bunta INO 2023-01-16               6900.64         6900.64   
4    Bunta INO 2023-01-17               5169.66         5169.66   

   total_distance  max_heart_rate  mean_heart_rate  min_heart_rate   max_vel  \
0      7501.64785           192.0       151.245632             0.0  26.24685   
1      6750.17156           188.0       146.024218             0.0  28.27672   
2      8739.70081           190.0       150.152842             0.0  30.02934   
3     10453.17890           166.0        94.788188             0.0  25.44576   
4      7472.99927           175.0       131.479998             0.0  27.68590   

   velocity_band1_total_distance  ...  ima_band3_left_count 

/tmp/ipykernel_18612/1214942399.py:75: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  date_array = daily_raw["date_"].dt.to_pydatetime()   # Python datetime に変換 → numpy


In [4]:
# ============================================================
# Season-aware ACWR（EWMA版）計算関数
#   - 長いオフ期間（gap_days > break_days）でリセット
# ============================================================

def add_acwr_features_season_aware(daily_df, var, acute=7, chronic=28, break_days=30):
    """
    選手ごとに日付順に並べ、
    ・日付差が break_days を超えたらシーズンを分割
    ・シーズンごとに EWMA を計算し、先頭 chronic 日は ACWR を NaN にする
    """
    df_acwr = daily_df.sort_values(["athlete_name", "date_"]).copy()

    # 初期化
    df_acwr[f"EWMA{acute}_{var}"] = np.nan
    df_acwr[f"EWMA{chronic}_{var}"] = np.nan
    df_acwr[f"ACWR_EWMA_{var}"] = np.nan
    df_acwr[f"EWMA_ratio_{var}"] = np.nan

    for athlete, idx in df_acwr.groupby("athlete_name").groups.items():
        sub = df_acwr.loc[idx].sort_values("date_").copy()

        # 日付のギャップ（日数）を計算
        gap_days = sub["date_"].diff().dt.days
        season_block = (gap_days > break_days).cumsum().fillna(0).astype(int)

        # シーズンブロックごとに EWMA を計算
        for block_id, b_idx in sub.groupby(season_block).groups.items():
            s = df_acwr.loc[b_idx, var]

            # 完全に0の期間などはスキップしてもよい
            if s.notna().sum() == 0:
                continue

            acute_series = s.ewm(span=acute, adjust=False).mean()
            chronic_series = s.ewm(span=chronic, adjust=False).mean()
            acwr_series = acute_series / chronic_series

            # ブロック内の先頭 chronic 日は ACWR を NaN
            acwr_series.iloc[:chronic] = np.nan

            df_acwr.loc[b_idx, f"EWMA{acute}_{var}"] = acute_series
            df_acwr.loc[b_idx, f"EWMA{chronic}_{var}"] = chronic_series
            df_acwr.loc[b_idx, f"ACWR_EWMA_{var}"] = acwr_series
            df_acwr.loc[b_idx, f"EWMA_ratio_{var}"] = acute_series / chronic_series

    return df_acwr

In [5]:
# ============================================================
# ACWRを追加する対象
# ============================================================

target_vars = ["total_distance", "total_player_load"]

# gap 分布のチェック（ちゃんとシーズン分割されそうか） ----------------
print("\n🔍 日付差（gap_days）の分布をざっくり確認")

gap_examples = []
for athlete, sub in daily_raw.groupby("athlete_name"):
    g = sub["date_"].sort_values().diff().dt.days.dropna()
    if not g.empty:
        gap_examples.append(g)

if gap_examples:
    gaps = pd.concat(gap_examples)
    print(gaps.describe())
else:
    print("gap が計算できる選手がいません")

# 実際に ACWR を付与 ---------------------------------------------
for var in target_vars:
    print(f"\n⚙ ACWR を計算中: {var}")
    daily_raw = add_acwr_features_season_aware(daily_raw, var)

print("\n✅ ACWR / EWMA 特徴量を daily_raw に追加完了！")
print(daily_raw.filter(regex="ACWR_EWMA|EWMA7_|EWMA28_").head())


🔍 日付差（gap_days）の分布をざっくり確認
count    12049.000000
mean         1.583617
std          3.165924
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        143.000000
Name: date_, dtype: float64

⚙ ACWR を計算中: total_distance

⚙ ACWR を計算中: total_player_load

✅ ACWR / EWMA 特徴量を daily_raw に追加完了！
   EWMA7_total_distance  EWMA28_total_distance  ACWR_EWMA_total_distance  \
0           7501.647850            7501.647850                       NaN   
1           7313.778777            7449.821899                       NaN   
2           7670.259286            7538.779065                       NaN   
3           8365.989189            7739.772157                       NaN   
4           8142.741709            7721.374027                       NaN   

   EWMA7_total_player_load  EWMA28_total_player_load  \
0               902.985100                902.985100   
1               864.034710                892.240165   
2               907.839628                902.

In [6]:
# ============================================================
# 保存（集約＋ACWR付きの日次データ）
# ============================================================
DAILY_DIR = "/workspace/data/02"
os.makedirs(DAILY_DIR, exist_ok=True)

daily_csv_path = f"{DAILY_DIR}/StatsAllGroup_full_players_daily.csv"
daily_parquet_path = f"{DAILY_DIR}/StatsAllGroup_full_players_daily.parquet"

daily_raw.to_csv(daily_csv_path, index=False)
daily_raw.to_parquet(daily_parquet_path, index=False)

print("💾 Saved daily data:")
print(" - CSV   :", daily_csv_path)
print(" - Parquet:", daily_parquet_path)


# %%
print("\n🔍 ACWR / EWMA 系列の列（存在チェック）")
acwr_cols = [c for c in daily_raw.columns if "ACWR_EWMA" in c or "EWMA7_" in c or "EWMA28_" in c]
print(acwr_cols)
print(f"📌 ACWR / EWMA 系列の列数: {len(acwr_cols)}")

print("\n🔍 ACWR / EWMA 先頭5行")
display(daily_raw[["athlete_name", "date_"] + acwr_cols].head())


# %%
print("\n📌 全列数:", len(daily_raw.columns))
daily_raw.info()

💾 Saved daily data:
 - CSV   : /workspace/data/02/StatsAllGroup_full_players_daily.csv
 - Parquet: /workspace/data/02/StatsAllGroup_full_players_daily.parquet

🔍 ACWR / EWMA 系列の列（存在チェック）
['EWMA7_total_distance', 'EWMA28_total_distance', 'ACWR_EWMA_total_distance', 'EWMA7_total_player_load', 'EWMA28_total_player_load', 'ACWR_EWMA_total_player_load']
📌 ACWR / EWMA 系列の列数: 6

🔍 ACWR / EWMA 先頭5行


,athlete_name,date_,EWMA7_total_distance,EWMA28_total_distance,ACWR_EWMA_total_distance,EWMA7_total_player_load,EWMA28_total_player_load,ACWR_EWMA_total_player_load
0,Bunta INO,2023-01-12,7501.647850,7501.647850,NaN,902.985100,902.985100,NaN
1,Bunta INO,2023-01-13,7313.778777,7449.821899,NaN,864.034710,892.240165,NaN
2,Bunta INO,2023-01-14,7670.259286,7538.779065,NaN,907.839628,902.379076,NaN
3,Bunta INO,2023-01-16,8365.989189,7739.772157,NaN,947.419483,913.674247,NaN
4,Bunta INO,2023-01-17,8142.741709,7721.374027,NaN,914.390800,906.890144,NaN



📌 全列数: 71
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12083 entries, 0 to 12082
Data columns (total 71 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   athlete_name                                    12083 non-null  object        
 1   date_                                           12083 non-null  datetime64[ns]
 2   session_duration_sec                            12083 non-null  float64       
 3   total_duration                                  12083 non-null  float64       
 4   total_distance                                  12083 non-null  float64       
 5   max_heart_rate                                  12083 non-null  float64       
 6   mean_heart_rate                                 12083 non-null  float64       
 7   min_heart_rate                                  12083 non-null  float64       
 8   max_vel                            

In [7]:
# ============================================================
# 特徴量分類（ACWR / Dynamic / Static）
# ============================================================

# --- 1. ACWR (EWMA/ACWR 系のみ) ---
acwr_cols = [
    c for c in daily_raw.columns
    if c.startswith("EWMA7_") 
    or c.startswith("EWMA28_")
    or c.startswith("ACWR_EWMA_")
    or c.startswith("EWMA_ratio_")
]

# --- 2. Dynamic（スパイク性の強い指標） ---
dynamic_cols = [
    "max_vel",
    "mean_heart_rate",
    "max_heart_rate",

    # 高速走行系
    "velocity_band4_total_distance",
    "velocity_band5_total_distance",
    "velocity_band6_total_distance",

    # 心拍高強度系
    "heart_rate_band3_total_duration",
    "heart_rate_band4_total_duration",

    # IMA / Accel- Decel 系（存在するものだけ使う）
    "acceleration_load",
    "ima_accel_high",
    "ima_decel_high",
    "ima_cod_left_high",
    "ima_cod_right_high",
]
dynamic_cols = [c for c in dynamic_cols if c in daily_raw.columns]

# --- total_distance / player_load は完全除外 ---
ban_cols = [
    "total_distance",
    "total_player_load"
]

# --- 3. Static（残りだが ban_cols を除外） ---
static_cols = [
    c for c in daily_raw.columns
    if c not in dynamic_cols
    and c not in acwr_cols
    and c not in ban_cols
    and c not in ["athlete_name", "date_", "md_offset", "md_phase", "is_match_day"]
]

# --- メタ列 ---
meta_cols = ["athlete_name", "date_", "md_offset", "md_phase", "is_match_day"]

# --- Split datasets ---
df_acwr    = daily_raw[meta_cols + acwr_cols].copy()
df_dynamic = daily_raw[meta_cols + dynamic_cols].copy()
df_static  = daily_raw[meta_cols + static_cols].copy()


In [8]:
# ===========================================
# 選手ごとのデータを保存（03 で使用）
# ===========================================
PLAYER_DIR = "/workspace/data/02/players"
os.makedirs(PLAYER_DIR, exist_ok=True)

for athlete, _ in daily_raw.groupby("athlete_name"):
    out_dir = f"{PLAYER_DIR}/{athlete}"
    os.makedirs(out_dir, exist_ok=True)

    df_dyn_p = df_dynamic[df_dynamic["athlete_name"] == athlete].copy()
    df_st_p  = df_static[df_static["athlete_name"] == athlete].copy()
    df_ac_p  = df_acwr[df_acwr["athlete_name"] == athlete].copy()

    df_dyn_p.to_parquet(f"{out_dir}/{athlete}_dynamic.parquet", index=False)
    df_st_p.to_parquet(f"{out_dir}/{athlete}_static.parquet", index=False)
    df_ac_p.to_parquet(f"{out_dir}/{athlete}_acwr.parquet", index=False)

print("💾 Saved per-player features under /workspace/data/02/players/")


💾 Saved per-player features under /workspace/data/02/players/
